<a href="https://colab.research.google.com/github/Haseebtanvir079/Advanced-DataScience/blob/main/Lab3_Part3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP7161 Lab Part 2

## Decision Tree Full Tutorial on "Adults" dataset

### 📌 Notebook Overview

In this notebook, we build a **Decision Tree classifier** using the Adult Income dataset.
The goal is to understand the **full machine learning pipeline**, from data loading and cleaning
to preprocessing, model building, and hyperparameter tuning.

Each section below explains **what we are doing**, **why it is necessary**, and **how it affects the model**.

### Importing Important Libraries

In this section, we import all the Python libraries required for this project.

These libraries serve different purposes:
- **NumPy** and **Pandas** are used for numerical computations and data manipulation.
- **Matplotlib** and **Seaborn** are used for data visualization.
- **Scikit-learn (sklearn)** provides tools for:
  - Machine learning models (Decision Tree)
  - Data splitting
  - Model evaluation
  - Hyperparameter tuning

Importing libraries at the beginning of the notebook is considered good practice, as it:
- Keeps the code organized
- Makes dependencies clear
- Helps avoid repeated imports later in the notebook

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Reading the Data

In this section, we load the Adult dataset into the notebook.

The dataset is read using **Pandas**, which converts the raw data into a DataFrame.
A DataFrame allows us to:
- Inspect the structure of the data
- Access rows and columns easily
- Apply cleaning and preprocessing operations efficiently

After loading the data, it is important to:
- Check the first few rows
- Understand the column names
- Verify that the data has been read correctly

In [ ]:
data = pd.read_csv(r'adult.csv')
data.sample(10)

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
42295,58,Private,284834,Assoc-acdm,12,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
5774,62,Private,155915,10th,6,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,<=50K
26894,50,Private,168539,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K
5899,29,Private,251526,Some-college,10,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,20,United-States,<=50K
35140,36,Local-gov,578377,Masters,14,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,40,United-States,<=50K
11751,26,Private,136309,Assoc-acdm,12,Never-married,Tech-support,Not-in-family,White,Male,0,0,50,United-States,<=50K
19110,36,Private,107302,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,60,United-States,>50K
40848,46,Private,84402,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Male,0,0,45,United-States,>50K
98,59,Private,272087,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,>50K
43343,76,Self-emp-not-inc,253408,Some-college,10,Widowed,Transport-moving,Not-in-family,White,Male,0,0,40,United-States,<=50K


### Cleaning the Data

Real-world datasets are often messy and contain issues such as:
- Missing values
- Extra spaces or inconsistent formatting
- Invalid or unknown entries

In this section, we clean the dataset to improve data quality.
Typical cleaning steps include:
- Handling missing or unknown values
- Removing unnecessary whitespace
- Ensuring consistency across categorical values

Data cleaning is a **critical step**, because:
- Machine learning models are sensitive to noisy data
- Poor-quality data can significantly reduce model performance

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational_num  48842 non-null  int64 
 5   marital_status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital_gain     48842 non-null  int64 
 11  capital_loss     48842 non-null  int64 
 12  hours_per_week   48842 non-null  int64 
 13  native_country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [ ]:
df_wk = data[data['workclass'] == '?']
df_wk

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
13,58,?,299831,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,35,United-States,<=50K
22,72,?,132015,7th-8th,4,Divorced,?,Not-in-family,White,Female,0,0,6,United-States,<=50K
35,65,?,191846,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48811,35,?,320084,Bachelors,13,Married-civ-spouse,?,Wife,White,Female,0,0,55,United-States,>50K
48812,30,?,33811,Bachelors,13,Never-married,?,Not-in-family,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
48820,71,?,287372,Doctorate,16,Married-civ-spouse,?,Husband,White,Male,0,0,10,United-States,>50K
48822,41,?,202822,HS-grad,9,Separated,?,Not-in-family,Black,Female,0,0,32,United-States,<=50K


In [ ]:
df_wk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2799 entries, 4 to 48823
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              2799 non-null   int64 
 1   workclass        2799 non-null   object
 2   fnlwgt           2799 non-null   int64 
 3   education        2799 non-null   object
 4   educational_num  2799 non-null   int64 
 5   marital_status   2799 non-null   object
 6   occupation       2799 non-null   object
 7   relationship     2799 non-null   object
 8   race             2799 non-null   object
 9   gender           2799 non-null   object
 10  capital_gain     2799 non-null   int64 
 11  capital_loss     2799 non-null   int64 
 12  hours_per_week   2799 non-null   int64 
 13  native_country   2799 non-null   object
 14  income           2799 non-null   object
dtypes: int64(6), object(9)
memory usage: 349.9+ KB


In [ ]:
data = data[data['workclass'] != '?']
data.sample(8)

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
47281,43,Private,475322,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,50,United-States,>50K
35268,48,Private,182541,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,15024,0,45,United-States,>50K
40880,26,Private,206307,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K
9394,63,Private,125954,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,44,United-States,>50K
13713,30,Private,118551,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,0,35,United-States,>50K
2972,35,Private,151835,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,50,United-States,>50K
40801,23,Private,57827,Bachelors,13,Never-married,Farming-fishing,Not-in-family,White,Male,0,0,40,United-States,<=50K
19307,28,Private,480861,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,45,United-States,<=50K


In [ ]:
data.shape

(46043, 15)

In [ ]:
df_categorical = data.select_dtypes(include=['object'])
df_categorical.apply(lambda x: x== '?').sum()

workclass           0
education           0
marital_status      0
occupation         10
relationship        0
race                0
gender              0
native_country    811
income              0
dtype: int64

In [ ]:
data = data[data['native_country'] != '?']
data = data[data['occupation'] != '?']
data.sample(8)

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
40406,50,Private,206862,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,65,United-States,>50K
20474,21,Private,370990,Some-college,10,Never-married,Sales,Own-child,White,Male,0,0,40,United-States,<=50K
39847,19,Private,101549,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,<=50K
10291,20,Private,196758,Assoc-acdm,12,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K
33721,35,Self-emp-not-inc,264627,11th,7,Divorced,Exec-managerial,Unmarried,White,Female,0,0,84,United-States,<=50K
19084,33,Private,164190,Masters,14,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K
22683,20,Private,49179,Some-college,10,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,35,United-States,<=50K
37399,20,Private,378546,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,25,United-States,<=50K


In [ ]:
data[data.occupation == '?']

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income


In [ ]:
data[data['native_country'] == '?']

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income


In [ ]:
df_categorical = data.select_dtypes(include=['object'])
df_categorical.apply(lambda x: x== '?').sum()

workclass         0
education         0
marital_status    0
occupation        0
relationship      0
race              0
gender            0
native_country    0
income            0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational_num  45222 non-null  int64 
 5   marital_status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital_gain     45222 non-null  int64 
 11  capital_loss     45222 non-null  int64 
 12  hours_per_week   45222 non-null  int64 
 13  native_country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


### Data Preprocessing

Before training a machine learning model, the data must be transformed into a format
that the algorithm can understand.

Decision Trees in scikit-learn require **numerical input**.
However, the Adult dataset contains several **categorical features**.

In this section, we prepare the data by:
- Separating numerical and categorical features
- Converting categorical features into numerical representations
- Making sure all features are suitable for model training

In [ ]:
categorical_data = data.select_dtypes(include=['object'])
categorical_data.head(10)

,workclass,education,marital_status,occupation,relationship,race,gender,native_country,income
0,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
5,Private,10th,Never-married,Other-service,Not-in-family,White,Male,United-States,<=50K
7,Self-emp-not-inc,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,>50K
8,Private,Some-college,Never-married,Other-service,Unmarried,White,Female,United-States,<=50K
9,Private,7th-8th,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States,<=50K
10,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
11,Federal-gov,Bachelors,Married-civ-spouse,Adm-clerical,Husband,White,Male,United-States,<=50K


In [ ]:
lab_enc = preprocessing.LabelEncoder()
categorical_data = categorical_data.apply(lab_enc.fit_transform)

categorical_data.head(10)

,workclass,education,marital_status,occupation,relationship,race,gender,native_country,income
0,2,1,4,6,3,2,1,38,0
1,2,11,2,4,0,4,1,38,0
2,1,7,2,10,0,4,1,38,1
3,2,15,2,6,0,2,1,38,1
5,2,0,4,7,1,4,1,38,0
7,4,14,2,9,0,4,1,38,1
8,2,15,4,7,4,4,0,38,0
9,2,5,2,2,0,4,1,38,0
10,2,11,2,6,0,4,1,38,1
11,0,9,2,0,0,4,1,38,0


### Dropping Categorical Columns from the Data

Here, we temporarily remove categorical columns from the dataset.

This step is useful because:
- Some models cannot directly process categorical data
- It allows us to handle numerical and categorical features separately
- It simplifies preprocessing and debugging

Categorical features will later be encoded and added back to the dataset
in a machine-readable numerical format.

In [ ]:
data = data.drop(categorical_data.columns, axis=1)
data.head()

,age,fnlwgt,educational_num,capital_gain,capital_loss,hours_per_week
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
5,34,198693,6,0,0,30


### Concatenating Both Numerical and Encoded Categorical Data

After encoding the categorical variables, we combine them back with
the numerical features.

This step creates a **final feature matrix** that:
- Contains only numerical values
- Preserves all original information from the dataset
- Can be safely used for training a machine learning model

At the end of this section, the dataset is fully prepared for modeling.

In [ ]:
data = pd.concat([data, categorical_data], axis=1)
data.head(10)

,age,fnlwgt,educational_num,capital_gain,capital_loss,hours_per_week,workclass,education,marital_status,occupation,relationship,race,gender,native_country,income
0,25,226802,7,0,0,40,2,1,4,6,3,2,1,38,0
1,38,89814,9,0,0,50,2,11,2,4,0,4,1,38,0
2,28,336951,12,0,0,40,1,7,2,10,0,4,1,38,1
3,44,160323,10,7688,0,40,2,15,2,6,0,2,1,38,1
5,34,198693,6,0,0,30,2,0,4,7,1,4,1,38,0
7,63,104626,15,3103,0,32,4,14,2,9,0,4,1,38,1
8,24,369667,10,0,0,40,2,15,4,7,4,4,0,38,0
9,55,104996,4,0,0,10,2,5,2,2,0,4,1,38,0
10,65,184454,9,6418,0,40,2,11,2,6,0,4,1,38,1
11,36,212465,13,0,0,40,0,9,2,0,0,4,1,38,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   age              45222 non-null  int64
 1   fnlwgt           45222 non-null  int64
 2   educational_num  45222 non-null  int64
 3   capital_gain     45222 non-null  int64
 4   capital_loss     45222 non-null  int64
 5   hours_per_week   45222 non-null  int64
 6   workclass        45222 non-null  int64
 7   education        45222 non-null  int64
 8   marital_status   45222 non-null  int64
 9   occupation       45222 non-null  int64
 10  relationship     45222 non-null  int64
 11  race             45222 non-null  int64
 12  gender           45222 non-null  int64
 13  native_country   45222 non-null  int64
 14  income           45222 non-null  int64
dtypes: int64(15)
memory usage: 5.5 MB


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   age              45222 non-null  int64
 1   fnlwgt           45222 non-null  int64
 2   educational_num  45222 non-null  int64
 3   capital_gain     45222 non-null  int64
 4   capital_loss     45222 non-null  int64
 5   hours_per_week   45222 non-null  int64
 6   workclass        45222 non-null  int64
 7   education        45222 non-null  int64
 8   marital_status   45222 non-null  int64
 9   occupation       45222 non-null  int64
 10  relationship     45222 non-null  int64
 11  race             45222 non-null  int64
 12  gender           45222 non-null  int64
 13  native_country   45222 non-null  int64
 14  income           45222 non-null  int64
dtypes: int64(15)
memory usage: 5.5 MB


In [ ]:
data.dtypes

age                int64
fnlwgt             int64
educational_num    int64
capital_gain       int64
capital_loss       int64
hours_per_week     int64
workclass          int64
education          int64
marital_status     int64
occupation         int64
relationship       int64
race               int64
gender             int64
native_country     int64
income             int64
dtype: object

In [ ]:
# you have one of the column which is of cateogory type
data["income"].head()

0    0
1    0
2    1
3    1
5    0
Name: income, dtype: int64

### Building the Model

In this section, we build a **Decision Tree Classifier**.

A Decision Tree works by:
- Splitting the data based on feature values
- Creating decision rules that lead to a final prediction

We also split the dataset into:
- **Training data**: used to train the model
- **Testing data**: used to evaluate model performance on unseen data

This separation is essential to:
- Avoid overfitting
- Measure how well the model generalizes

In [ ]:
X = data.drop('income', axis=1)
Y = data['income']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3)

In [ ]:
X_train.shape

(31655, 14)

In [ ]:
Y_train.shape

(31655,)

In [ ]:
Y_test.shape

(13567,)

In [ ]:
X_test.shape

(13567, 14)

In [ ]:
dt_default = DecisionTreeClassifier(max_depth=5)
dt_default.fit(X_train, Y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",5
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current 

In [ ]:
Y_pred_default = dt_default.predict(X_test)

In [ ]:
print(classification_report(Y_test, Y_pred_default))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90     10196
           1       0.77      0.53      0.63      3371

    accuracy                           0.84     13567
   macro avg       0.81      0.74      0.76     13567
weighted avg       0.84      0.84      0.83     13567



In [ ]:
print(confusion_matrix(Y_test, Y_pred_default))

[[9661  535]
 [1584 1787]]


In [ ]:
print(accuracy_score(Y_test, Y_pred_default))

0.84381219134665


### Tuning with max_depth using GridSearchCV

The `max_depth` parameter controls how deep the decision tree can grow.

- A very deep tree may overfit the training data
- A shallow tree may underfit and miss important patterns

In this section, we use **GridSearchCV** to:
- Try different values of `max_depth`
- Perform cross-validation
- Automatically select the best value based on accuracy

GridSearchCV helps us find optimal hyperparameters in a systematic way.

In [ ]:
n_folds = 5
parameters = {'max_depth': range(1, 40)}

In [ ]:
dtree = DecisionTreeClassifier(criterion='gini')
tree = GridSearchCV(dtree, parameters, cv=n_folds, scoring='accuracy')
tree.fit(X_train, Y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'max_depth': range(1, 40)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate param

In [ ]:
Y_pred_dtree = tree.predict(X_test)

In [ ]:
print(accuracy_score(Y_test, Y_pred_dtree))

0.8491191862607799


In [ ]:
scores = tree.cv_results_
pd.DataFrame(scores).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011468,0.002440,0.001823,0.001051,1,{'max_depth': 1},0.752488,0.752488,0.752488,0.752330,0.752330,0.752425,0.000077,39
1,0.010618,0.000751,0.000660,0.000049,2,{'max_depth': 2},0.819460,0.822619,0.821197,0.817722,0.826252,0.821450,0.002910,17
2,0.013862,0.000113,0.000624,0.000008,3,{'max_depth': 3},0.832728,0.835729,0.835571,0.833044,0.841415,0.835697,0.003117,12
3,0.017817,0.000074,0.000645,0.000022,4,{'max_depth': 4},0.842679,0.843943,0.844258,0.842995,0.849155,0.844606,0.002348,8
4,0.021566,0.000177,0.000676,0.000053,5,{'max_depth': 5},0.847733,0.847891,0.849313,0.846312,0.855157,0.849281,0.003088,6


### Tuning with min_samples_leaf

The `min_samples_leaf` parameter specifies:
- The minimum number of samples required in a leaf node

This parameter helps:
- Prevent overfitting
- Create more stable and generalizable decision rules

By tuning this parameter, we control how specific or general the tree's decisions are.

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

In [ ]:
n_folds = 5
parameters = {'min_samples_leaf': range(5, 200, 20)}

In [ ]:
dtree_msl = DecisionTreeClassifier(criterion='gini')
tree_msl = GridSearchCV(dtree, parameters, cv=n_folds, scoring='accuracy')
tree_msl.fit(X_train, Y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'min_samples_leaf': range(5, 200, 20)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and can

In [ ]:
Y_pred_dtree_msl = tree_msl.predict(X_test)

In [ ]:
print(accuracy_score(Y_test, Y_pred_dtree_msl))

0.8497825606250461


In [ ]:
scores_msl = tree_msl.cv_results_
s = pd.DataFrame(scores)
s.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011468,0.002440,0.001823,0.001051,1,{'max_depth': 1},0.752488,0.752488,0.752488,0.752330,0.752330,0.752425,0.000077,39
1,0.010618,0.000751,0.000660,0.000049,2,{'max_depth': 2},0.819460,0.822619,0.821197,0.817722,0.826252,0.821450,0.002910,17
2,0.013862,0.000113,0.000624,0.000008,3,{'max_depth': 3},0.832728,0.835729,0.835571,0.833044,0.841415,0.835697,0.003117,12
3,0.017817,0.000074,0.000645,0.000022,4,{'max_depth': 4},0.842679,0.843943,0.844258,0.842995,0.849155,0.844606,0.002348,8
4,0.021566,0.000177,0.000676,0.000053,5,{'max_depth': 5},0.847733,0.847891,0.849313,0.846312,0.855157,0.849281,0.003088,6


In [ ]:
list(s.columns)

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_max_depth',
 'params',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'mean_test_score',
 'std_test_score',
 'rank_test_score']

### Tuning with min_samples_split

The `min_samples_split` parameter defines:
- The minimum number of samples required to split an internal node

Larger values:
- Make the tree more conservative
- Reduce model complexity

Smaller values:
- Allow more splits
- Increase the risk of overfitting

This section explores how this parameter affects model performance.

In [ ]:
from sklearn.model_selection import KFold, GridSearchCV

In [ ]:
n_folds = 5
parameters = {'min_samples_split': range(5, 200, 20)}

In [ ]:
dtree_mss = DecisionTreeClassifier(criterion='gini')
tree_mss = GridSearchCV(dtree, parameters, cv=n_folds, scoring='accuracy')
tree_mss.fit(X_train, Y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'min_samples_split': range(5, 200, 20)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and ca

In [ ]:
Y_pred_dtree_mss = tree_mss.predict(X_test)

In [ ]:
print(accuracy_score(Y_test, Y_pred_dtree_mss))

0.850961892828186


In [ ]:
scores_mss = tree_mss.cv_results_
mss = pd.DataFrame(scores)
mss.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.011468,0.002440,0.001823,0.001051,1,{'max_depth': 1},0.752488,0.752488,0.752488,0.752330,0.752330,0.752425,0.000077,39
1,0.010618,0.000751,0.000660,0.000049,2,{'max_depth': 2},0.819460,0.822619,0.821197,0.817722,0.826252,0.821450,0.002910,17
2,0.013862,0.000113,0.000624,0.000008,3,{'max_depth': 3},0.832728,0.835729,0.835571,0.833044,0.841415,0.835697,0.003117,12
3,0.017817,0.000074,0.000645,0.000022,4,{'max_depth': 4},0.842679,0.843943,0.844258,0.842995,0.849155,0.844606,0.002348,8
4,0.021566,0.000177,0.000676,0.000053,5,{'max_depth': 5},0.847733,0.847891,0.849313,0.846312,0.855157,0.849281,0.003088,6


In [ ]:
list(mss.columns)

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_max_depth',
 'params',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'mean_test_score',
 'std_test_score',
 'rank_test_score']

### Tuning with All Parameters at Once

In this section, we tune multiple hyperparameters simultaneously.

By combining:
- `max_depth`
- `min_samples_leaf`
- `min_samples_split`

we allow GridSearchCV to:
- Explore a wider hyperparameter space
- Find the best overall configuration
- Improve model robustness and performance

This approach usually leads to better results than tuning parameters individually.

In [ ]:
param_grid = {
    'max_depth': range(5, 150, 50),
    'min_samples_leaf': range(50, 150, 50),
    'min_samples_split': range(50, 150, 50),
    'criterion': ['entropy', 'gini']
}

In [ ]:
n_folds = 5

In [ ]:
dtree_param_grid = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator=dtree_param_grid,
                           param_grid=param_grid,
                          cv=n_folds, verbose=1)

In [ ]:
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'criterion': ['entropy', 'gini'], 'max_depth': range(5, 150, 50), 'min_samples_leaf': range(50, 150, 50), 'min_samples_split': range(50, 150, 50)}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for eac

In [ ]:
cv_results = pd.DataFrame(grid_search.cv_results_)

In [ ]:
cv_results.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029134,0.009843,0.000890,0.000281,entropy,5,50,50,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.842679,0.846154,0.843943,0.837782,0.849787,0.844069,0.003963,21
1,0.022585,0.000110,0.000641,0.000007,entropy,5,50,100,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.842679,0.846154,0.843943,0.837782,0.849787,0.844069,0.003963,21
2,0.022290,0.000187,0.000639,0.000025,entropy,5,100,50,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.842679,0.846154,0.843943,0.837782,0.849471,0.844006,0.003872,23
3,0.022453,0.000140,0.000601,0.000012,entropy,5,100,100,"{'criterion': 'entropy', 'max_depth': 5, 'min_...",0.842679,0.846154,0.843943,0.837782,0.849471,0.844006,0.003872,23
4,0.042975,0.000686,0.000814,0.000030,entropy,55,50,50,"{'criterion': 'entropy', 'max_depth': 55, 'min...",0.846312,0.850103,0.849471,0.844732,0.854841,0.849092,0.003492,13
5,0.043817,0.001654,0.000895,0.000110,entropy,55,50,100,"{'criterion': 'entropy', 'max_depth': 55, 'min...",0.846312,0.850103,0.849471,0.844732,0.854841,0.849092,0.003492,13
6,0.039174,0.000334,0.000807,0.000029,entropy,55,100,50,"{'criterion': 'entropy', 'max_depth': 55, 'min...",0.845048,0.850419,0.848049,0.847102,0.857211,0.849566,0.004194,9
7,0.039198,0.000539,0.000820,0.000008,entropy,55,100,100,"{'criterion': 'entropy', 'max_depth': 55, 'min...",0.845048,0.850419,0.848049,0.847102,0.857211,0.849566,0.004194,9
8,0.042931,0.000729,0.000804,0.000020,entropy,105,50,50,"{'criterion': 'entropy', 'max_depth': 105, 'mi...",0.846312,0.850103,0.849471,0.844732,0.854841,0.849092,0.003492,13
9,0.043139,0.000843,0.000792,0.000026,entropy,105,50,100,"{'criterion': 'entropy', 'max_depth': 105, 'mi...",0.846312,0.850103,0.849471,0.844732,0.854841,0.849092,0.003492,13


In [ ]:
print('Best Accuracy is: ', grid_search.best_score_)
print('\n\n')
print(grid_search.best_estimator_)

Best Accuracy is:  0.851271521086716



DecisionTreeClassifier(max_depth=55, min_samples_leaf=50, min_samples_split=50)


### Checking the Best Accuracy from the Best Estimator

Finally, we evaluate the best model selected by GridSearchCV.

In this section, we:
- Retrieve the best estimator
- Use it to make predictions on the test data
- Measure the final accuracy

This step confirms:
- How well the tuned model performs
- Whether hyperparameter tuning improved the results

In [ ]:
clf_best = DecisionTreeClassifier(criterion='gini',
                                 max_depth=55,
                                 min_samples_leaf=50,
                                 min_samples_split=100)
clf_best.fit(X_train, Y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",55
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",100
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",50
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the curr

In [ ]:
ypred_best = clf_best.predict(X_test)

In [ ]:
print(accuracy_score(Y_test, ypred_best))

0.848603228421906


## 📚 References and Further Reading

The following resources are recommended for students who want to explore
Decision Trees and data preprocessing in more depth:

### Core Concepts
- Scikit-learn Documentation – Decision Trees  
  https://scikit-learn.org/stable/modules/tree.html

- Introduction to Machine Learning with Python (Chapter on Decision Trees)  
  https://www.oreilly.com/library/view/introduction-to-machine/9781449369880/

### Data Preprocessing
- Scikit-learn – Preprocessing Categorical Features  
  https://scikit-learn.org/stable/modules/preprocessing.html

- Handling Categorical Data in Machine Learning  
  https://machinelearningmastery.com/handle-categorical-data-machine-learning/

### Model Selection and Hyperparameter Tuning
- GridSearchCV Documentation  
  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

- Cross-Validation Explained  
  https://scikit-learn.org/stable/modules/cross_validation.html